# Carl Klein - Biophysics 435 Final Project: Predicting Base Stacking and SASA for RNA

In [0]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from time import time
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report

import pandas as pd
import io
import requests
import warnings
from scipy.stats import randint as sp_randint
from scipy.stats import expon as sp_expon

In [0]:
# Global variable 
NUMBER_CHEMICAL_SHIFT_TYPE = 19

def get_cs_all(cs_all, id = "2KOC"):
  '''    
    This function gets chemical shifts for a particular RNA. 
    Assumes each RNA has a unique id  
  '''
  return(cs_all[(cs_all.id == id)])

def get_cs_residues(cs_i, resid, dummy = 0):
  '''    
    This function return an array containing the chemical shifts for a particular residues in an RNA.    
  '''
  cs_tmp = cs_i[(cs_i.resid == resid)].drop(['id', 'resid', 'resname', 'stacking', 'orientation', 'pseudoknot', 'base_pairing', 'sugar_puckering'], axis=1)
  info_tmp = cs_i[(cs_i.resid == resid)]
  if (cs_tmp.shape[0] != 1):
     return(dummy*np.ones(shape=(1, NUMBER_CHEMICAL_SHIFT_TYPE)))
  else:
     return(cs_tmp.values)
    
def get_resnames(cs_i, resid, dummy = "UNK"):
  '''    
    This function returns the residue name for specified residue (resid)
  '''
  cs_tmp = cs_i[(cs_i.resid == resid)]  
  if (cs_tmp.shape[0] != 1):
     return(dummy)
  else:
     return(cs_tmp['resname'].values[0])

def get_cs_features(cs_i, resid, neighbors=1):
  '''    
  This function chemical shifts and resnames for residue (resid) and its neighbors        

  '''
  cs = []
  resnames = []
  for i in range(resid-neighbors, resid+neighbors+1):
    cs.append(get_cs_residues(cs_i, i))
    resnames.append(get_resnames(cs_i, i))
  return(resnames, np.array(cs))

def get_columns_names(neighbors = 3, chemical_shift_types = 19):
  '''
    
    Helper function that writes out the required column names
    
  '''

  columns = ['id', 'resname', 'resid', 'stacking', 'orientation', 'pseudoknot', 'base_pairing', 'sugar_puckering']
  for i in range(0, neighbors*chemical_shift_types):
    columns.append(i)
  return(columns)

def write_out_resname(neighbors=1):
  '''
  
    Helper function that writes out the column names associated resnames for a given residue and its neighbors
    
  '''  
  colnames = []
  for i in range(1-neighbors-1, neighbors+1):
    if i < 0: 
      colnames.append('R%s'%i)
    elif i > 0: 
      colnames.append('R+%s'%i)
    else: 
      colnames.append('R')
  return(colnames)    


def get_cs_features_rna(cs, neighbors=1, retain = ['id', 'stacking', 'resid', 'orientation', 'pseudoknot', 'base_pairing', 'sugar_puckering']):
  '''    
    This function generates the complete required data frame an RNA    
  '''
  all_features = []
  all_resnames = []
  for resid in cs['resid'].unique():
    resnames, features = get_cs_features(cs, resid, neighbors)
    all_features.append(features.flatten())
    all_resnames.append(resnames)

  all_resnames = pd.DataFrame(all_resnames, dtype='object', columns = write_out_resname(neighbors))
  all_features = pd.DataFrame(all_features, dtype='object')
  info = pd.DataFrame(cs[retain].values, dtype='object', columns = retain)
  return(pd.concat([info, all_resnames, all_features], axis=1))

In [0]:
# Start: your code
def get_cs_features_rna_all(cs, neighbors = 2, retain = ['id', 'stacking', 'resid', 'orientation', 'pseudoknot', 'base_pairing', 'sugar_puckering']):
  ids = cs['id'].unique()
  for i,id in enumerate(ids):
    if i == 0:
      cs_new = get_cs_features_rna(get_cs_all(cs, id), neighbors)
    else:
      cs_new = cs_new.append(get_cs_features_rna(get_cs_all(cs, id), neighbors), sort = False)
  
  # End: your code
  return(cs_new)

In [4]:
!apt-get -qq install -y python-rdkit librdkit1 rdkit-data

Selecting previously unselected package fonts-freefont-ttf.
(Reading database ... 134985 files and directories currently installed.)
Preparing to unpack .../fonts-freefont-ttf_20120503-7_all.deb ...
Unpacking fonts-freefont-ttf (20120503-7) ...
Selecting previously unselected package librdkit1.
Preparing to unpack .../librdkit1_201603.5+dfsg-1ubuntu1_amd64.deb ...
Unpacking librdkit1 (201603.5+dfsg-1ubuntu1) ...
Selecting previously unselected package rdkit-data.
Preparing to unpack .../rdkit-data_201603.5+dfsg-1ubuntu1_all.deb ...
Unpacking rdkit-data (201603.5+dfsg-1ubuntu1) ...
Selecting previously unselected package python-rdkit.
Preparing to unpack .../python-rdkit_201603.5+dfsg-1ubuntu1_amd64.deb ...
Unpacking python-rdkit (201603.5+dfsg-1ubuntu1) ...
Setting up rdkit-data (201603.5+dfsg-1ubuntu1) ...
Setting up fonts-freefont-ttf (20120503-7) ...
Setting up librdkit1 (201603.5+dfsg-1ubuntu1) ...
Setting up python-rdkit (201603.5+dfsg-1ubuntu1) ...
Processing triggers for libc-bi

In [5]:
!pip install -q joblib pandas sklearn tensorflow pillow deepchem

     |████████████████████████████████| 3.9MB 2.8MB/s 


In [6]:
import numpy as np
import tensorflow as tf

In [0]:
#get the dataset and store it in a Pandas Dataframe
url="https://drive.google.com/uc?id=1e-SHtWDtg4mD_th3_4Jmq9r1iiQC32wT"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')), sep= " ")

In [10]:
#dataset prior to one-hot encoding
cs_all = get_cs_features_rna_all(c, neighbors = 0)

cs_all

,id,stacking,resid,orientation,pseudoknot,base_pairing,sugar_puckering,R,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1A60,stack,1,anti,non-pseudoknotted,0,~C3'-endo,GUA,0,92.7,74.9,72,82.5,65.4,153.6,109.8,157,142.1,5.65,4.5,4.12,4.23,7.469,6.124,4.07,3.99,8.413,8.03
1,1A60,stack,2,anti,non-pseudoknotted,0,~C3'-endo,GUA,1,91.9,75.7,72.4,82.3,66.1,155.207,118.377,160.7,141.4,5.88,4.6,4.5,4.31,7.83,5.549,4.2,4.15,7.871,7.49
2,1A60,stack,3,anti,non-pseudoknotted,0,~C3'-endo,GUA,2,92.2,76.1,72.3,84.7,66,154.079,120.289,160.7,141.2,5.77,4.5,4.09,4.19,7.776,5.651,4.1,4.08,7.904,7.24
3,1A60,stack,4,anti,non-pseudoknotted,0,~C3'-endo,ADE,3,92.2,75,72,81.2,65.8,153.7,118.8,157.085,139.1,5.96,4.57,4.53,4.54,7.52,5.932,4.28,4.11,8.413,7.68
4,1A60,stack,5,anti,non-pseudoknotted,0,~C3'-endo,GUA,4,92.2,74.8,71.8,85.7,65.4,155.07,118.2,161.165,136.1,5.63,4.43,4.4,4.18,7.793,5.988,4.18,4.07,8.522,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,5WQ1,stack,19,anti,non-pseudoknotted,0,~C3'-endo,URA,3347,93.656,71.924,72.503,80.33,64.042,152.055,103.523,141.5,139.8,5.494,4.558,4.422,4.561,7.4,5.216,4.469,4.085,7.618,7.914
19,5WQ1,stack,20,anti,non-pseudoknotted,0,~C3'-endo,GUA,3348,92.923,75.273,73.402,82.035,65.331,155.24,118.7,161.6,136.271,5.779,4.496,4.583,4.502,7.947,5.932,4.504,4.127,7.93,7.702
20,5WQ1,stack,21,anti,non-pseudoknotted,0,~C3'-endo,URA,3349,93.692,75.31,73.04,82.92,64.482,153.472,102.537,141.916,141.418,5.537,4.481,4.25,4.291,7.043,5.094,4.478,4.08,7.823,8.3
21,5WQ1,stack,22,anti,non-pseudoknotted,0,~C3'-endo,CYT,3350,94.159,75.698,69.764,81.554,64.533,158.72,97.531,141.939,140.551,5.587,4.233,4.471,4.413,7.046,5.634,4.526,4.072,7.894,7.706


In [0]:
#encode the data in the stacking column
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
cs_all['stacking']= labelencoder.fit_transform(cs_all['stacking'])

In [12]:
#cs_all dataset with encoded stacking data
cs_all.head(n=10)

,id,stacking,resid,orientation,pseudoknot,base_pairing,sugar_puckering,R,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1A60,1,1,anti,non-pseudoknotted,0,~C3'-endo,GUA,0,92.7,74.9,72,82.5,65.4,153.6,109.8,157,142.1,5.65,4.5,4.12,4.23,7.469,6.124,4.07,3.99,8.413,8.03
1,1A60,1,2,anti,non-pseudoknotted,0,~C3'-endo,GUA,1,91.9,75.7,72.4,82.3,66.1,155.207,118.377,160.7,141.4,5.88,4.6,4.5,4.31,7.83,5.549,4.2,4.15,7.871,7.49
2,1A60,1,3,anti,non-pseudoknotted,0,~C3'-endo,GUA,2,92.2,76.1,72.3,84.7,66,154.079,120.289,160.7,141.2,5.77,4.5,4.09,4.19,7.776,5.651,4.1,4.08,7.904,7.24
3,1A60,1,4,anti,non-pseudoknotted,0,~C3'-endo,ADE,3,92.2,75,72,81.2,65.8,153.7,118.8,157.085,139.1,5.96,4.57,4.53,4.54,7.52,5.932,4.28,4.11,8.413,7.68
4,1A60,1,5,anti,non-pseudoknotted,0,~C3'-endo,GUA,4,92.2,74.8,71.8,85.7,65.4,155.07,118.2,161.165,136.1,5.63,4.43,4.4,4.18,7.793,5.988,4.18,4.07,8.522,7
5,1A60,0,6,anti,non-pseudoknotted,1,~C3'-endo,CYT,5,93.1,76,71.8,82.2,64,159.7,96.5,141.7,141.851,5.64,4.17,4.42,4.32,7.4,5.11,4.32,4.17,7.48,8.169
6,1A60,0,7,anti,non-pseudoknotted,1,~C3'-endo,URA,6,92.4,74.8,73.5,83.1,65.5,150.76,104.5,143.7,140.758,5.88,4.35,4.65,4.38,6.077,5.82,4.14,4.13,7.89,8.291
7,1A60,0,8,anti,non-pseudoknotted,1,~C2'-endo,CYT,7,93.4,75.1,72.5,82.3,64.2,155.24,97.7,143.1,138.179,5.5,4.43,4.3,4.29,7.33,5.76,4.08,3.98,7.74,7.6
8,1A60,1,9,anti,non-pseudoknotted,1,~C3'-endo,ADE,8,91.4,75.4,73.3,82.4,66,154.9,118.176,158.057,141.1,6.02,4.67,4.87,4.39,7.86,6.081,4.3,4.26,7.968,8.05
9,1A60,1,10,anti,non-pseudoknotted,1,~C3'-endo,ADE,9,91.5,75,73.6,82.9,66.2,154.9,120.239,157.085,140,5.36,4.35,4.52,4.49,8.04,6.081,4.35,4.33,7.93,7.66


In [13]:
#original dataset
c

,Unnamed: 0,resid,id,resname,C1p,C2p,C3p,C4p,C5p,C2,C5,C6,C8,H1p,H2p,H3p,H4p,H2,H5,H5p,H5pp,H6,H8,base_pairing,orientation,sugar_puckering,stacking,pseudoknot
0,0,1,1A60,GUA,92.700,74.900,72.000,82.500,65.400,153.600,109.800,157.000,142.100,5.650,4.500,4.120,4.230,7.469,6.124,4.070,3.990,8.413,8.0300,0,anti,~C3'-endo,stack,non-pseudoknotted
1,1,2,1A60,GUA,91.900,75.700,72.400,82.300,66.100,155.207,118.377,160.700,141.400,5.880,4.600,4.500,4.310,7.830,5.549,4.200,4.150,7.871,7.4900,0,anti,~C3'-endo,stack,non-pseudoknotted
2,2,3,1A60,GUA,92.200,76.100,72.300,84.700,66.000,154.079,120.289,160.700,141.200,5.770,4.500,4.090,4.190,7.776,5.651,4.100,4.080,7.904,7.2400,0,anti,~C3'-endo,stack,non-pseudoknotted
3,3,4,1A60,ADE,92.200,75.000,72.000,81.200,65.800,153.700,118.800,157.085,139.100,5.960,4.570,4.530,4.540,7.520,5.932,4.280,4.110,8.413,7.6800,0,anti,~C3'-endo,stack,non-pseudoknotted
4,4,5,1A60,GUA,92.200,74.800,71.800,85.700,65.400,155.070,118.200,161.165,136.100,5.630,4.430,4.400,4.180,7.793,5.988,4.180,4.070,8.522,7.0000,0,anti,~C3'-endo,stack,non-pseudoknotted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3063,3347,19,5WQ1,URA,93.656,71.924,72.503,80.330,64.042,152.055,103.523,141.500,139.800,5.494,4.558,4.422,4.561,7.400,5.216,4.469,4.085,7.618,7.9140,0,anti,~C3'-endo,stack,non-pseudoknotted
3064,3348,20,5WQ1,GUA,92.923,75.273,73.402,82.035,65.331,155.240,118.700,161.600,136.271,5.779,4.496,4.583,4.502,7.947,5.932,4.504,4.127,7.930,7.7020,0,anti,~C3'-endo,stack,non-pseudoknotted
3065,3349,21,5WQ1,URA,93.692,75.310,73.040,82.920,64.482,153.472,102.537,141.916,141.418,5.537,4.481,4.250,4.291,7.043,5.094,4.478,4.080,7.823,8.3000,0,anti,~C3'-endo,stack,non-pseudoknotted
3066,3350,22,5WQ1,CYT,94.159,75.698,69.764,81.554,64.533,158.720,97.531,141.939,140.551,5.587,4.233,4.471,4.413,7.046,5.634,4.526,4.072,7.894,7.7060,0,anti,~C3'-endo,stack,non-pseudoknotted


# MLP model (modified from base-pairing version)

In [0]:
def create_training_testing(cs, leave_out = "2KOC", target_name = 'stacking', neighbors = 0, drop_names = ['id', 'base_pairing', 'resid', 'orientation', 'sugar_puckering', 'pseudoknot']):
  '''    
    This function creates a training and testing set using leave one out    
  '''
  
  # drop extraneous data  
  drop_names = drop_names + list(write_out_resname(neighbors))  
  
  # does not contain leave_out
  train = cs[(cs.id != leave_out)]
  trainX = train.drop(drop_names, axis=1)
  trainy = train[target_name]
 
  # only contains leave_out
  test = cs[(cs.id == leave_out)]
  testX = test.drop(drop_names, axis=1)
  testy = test[target_name]
  
  # return training and testing data
  return(trainX.values, trainy.values, testX.values, testy.values)

In [15]:
NEIGHBORS = 0
id = '2KOC'
trainX, trainy, testX, testy = create_training_testing(cs_all, leave_out = id, neighbors = NEIGHBORS)
print("[INFO]: created training and testing data structures")

# setup scaler
scaler = StandardScaler()
scaler.fit(trainX)

# transform input
trainX_scaled = scaler.transform(trainX)
testX_scaled = scaler.transform(testX)
print("[INFO]: scaled the features")

[INFO]: created training and testing data structures
[INFO]: scaled the features


**MLP Classifier model w/o optimization**

In [0]:
#split the data into train/test
from sklearn.model_selection import train_test_split
X=cs_all[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]] #Chemical shift data
y=cs_all['stacking'] #one-hot encoded stacking data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [0]:
#Scale the features
scaler = StandardScaler()
scaler.fit(X_train)
trainX_scaled = scaler.transform(X_train)
testX_scaled = scaler.transform(X_test)

In [18]:
#fit and evaluate a baseline MLP model w/o optimization
from sklearn.neural_network import MLPClassifier
classifier = MLPClassifier(max_iter=100)
classifier.fit(trainX_scaled, y_train)
y_pred = classifier.predict(testX_scaled)
print('score report for predicting stacking')
print(classification_report(y_test, y_pred))

score report for predicting stacking
              precision    recall  f1-score   support

           0       0.62      0.13      0.21       119
           1       0.88      0.99      0.93       802

    accuracy                           0.88       921
   macro avg       0.75      0.56      0.57       921
weighted avg       0.85      0.88      0.84       921



/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


**Hyperparameter Optimization**

In [0]:
parameter_space = {
    'hidden_layer_sizes': [(50,50,50,50), (50,50,50), (50,50), (50,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam', 'lbfgs'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

from scipy.stats import randint as sp_randint
from scipy.stats import expon as sp_expon

min_size, max_size = 5, 100
parameter_space_distribution = {
    'hidden_layer_sizes': [(sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size)), (sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size)), (sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size)), (sp_randint.rvs(min_size, max_size),)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam', 'lbfgs'],
    'alpha': sp_expon(scale=.01),
    'learning_rate': ['constant','adaptive'],
    'learning_rate_init': sp_expon(scale=.001),
}

In [20]:
n_iter_search = 10
random_search = RandomizedSearchCV(classifier, param_distributions=parameter_space_distribution, n_iter=n_iter_search, cv=4, verbose = 5)
random_search.fit(trainX_scaled, y_train)


Fitting 4 folds for each of 10 candidates, totalling 40 fits
[CV] activation=tanh, alpha=0.008873749889043728, hidden_layer_sizes=(5, 68, 37), learning_rate=adaptive, learning_rate_init=0.000626839428279053, solver=sgd 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s remaining:    0.0s


[CV]  activation=tanh, alpha=0.008873749889043728, hidden_layer_sizes=(5, 68, 37), learning_rate=adaptive, learning_rate_init=0.000626839428279053, solver=sgd, score=0.864, total=   1.8s
[CV] activation=tanh, alpha=0.008873749889043728, hidden_layer_sizes=(5, 68, 37), learning_rate=adaptive, learning_rate_init=0.000626839428279053, solver=sgd 


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.6s remaining:    0.0s


[CV]  activation=tanh, alpha=0.008873749889043728, hidden_layer_sizes=(5, 68, 37), learning_rate=adaptive, learning_rate_init=0.000626839428279053, solver=sgd, score=0.866, total=   1.8s
[CV] activation=tanh, alpha=0.008873749889043728, hidden_layer_sizes=(5, 68, 37), learning_rate=adaptive, learning_rate_init=0.000626839428279053, solver=sgd 


...

[CV]  activation=tanh, alpha=0.01876598928124001, hidden_layer_sizes=(5, 68, 37), learning_rate=constant, learning_rate_init=0.00024155141311938758, solver=sgd, score=0.866, total=   1.8s
[CV] activation=tanh, alpha=0.01876598928124001, hidden_layer_sizes=(5, 68, 37), learning_rate=constant, learning_rate_init=0.00024155141311938758, solver=sgd 


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:  1.1min finished


[CV]  activation=tanh, alpha=0.01876598928124001, hidden_layer_sizes=(5, 68, 37), learning_rate=constant, learning_rate_init=0.00024155141311938758, solver=sgd, score=0.866, total=   1.8s


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


RandomizedSearchCV(cv=4, error_score='raise-deprecating',
                   estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                           batch_size='auto', beta_1=0.9,
                                           beta_2=0.999, early_stopping=False,
                                           epsilon=1e-08,
                                           hidden_layer_sizes=(100,),
                                           learning_rate='constant',
                                           learning_rate_init=0.001,
                                           max_iter=100, momentum=0.9,
                                           n_iter_no_change=10,
                                           nesterovs_momentum=True, power_t=0.5,
                                           rand...
                                        'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7efee3ad9550>,
                                        'hidden_layer_sizes

In [21]:
# Best parameter set
print('Best parameters found:\n', random_search.best_params_)

# All results
means = random_search.cv_results_['mean_test_score']
stds = random_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, random_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

Best parameters found:
 {'activation': 'tanh', 'alpha': 0.018940922168679448, 'hidden_layer_sizes': (70,), 'learning_rate': 'adaptive', 'learning_rate_init': 0.0007713997545576555, 'solver': 'adam'}
0.865 (+/-0.001) for {'activation': 'tanh', 'alpha': 0.008873749889043728, 'hidden_layer_sizes': (5, 68, 37), 'learning_rate': 'adaptive', 'learning_rate_init': 0.000626839428279053, 'solver': 'sgd'}
0.809 (+/-0.018) for {'activation': 'relu', 'alpha': 0.0032312890846254716, 'hidden_layer_sizes': (27, 92), 'learning_rate': 'adaptive', 'learning_rate_init': 0.0008775066052608679, 'solver': 'lbfgs'}
0.857 (+/-0.013) for {'activation': 'tanh', 'alpha': 0.005846314476543495, 'hidden_layer_sizes': (62, 20, 80, 58), 'learning_rate': 'adaptive', 'learning_rate_init': 0.0005077118290684212, 'solver': 'adam'}
0.870 (+/-0.005) for {'activation': 'tanh', 'alpha': 0.018940922168679448, 'hidden_layer_sizes': (70,), 'learning_rate': 'adaptive', 'learning_rate_init': 0.0007713997545576555, 'solver': 'adam

In [22]:
#score report for model optimized via random search
y_true, y_pred = y_test , random_search.predict(testX_scaled)
print('Results on the test set:')
print(classification_report(y_true, y_pred))

Results on the test set:
              precision    recall  f1-score   support

           0       0.76      0.11      0.19       119
           1       0.88      1.00      0.94       802

    accuracy                           0.88       921
   macro avg       0.82      0.55      0.56       921
weighted avg       0.87      0.88      0.84       921



# Random Forest Model

In [0]:
#Split data into train/test
from sklearn.model_selection import train_test_split
X=cs_all[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]] #Chemical shift data
y=cs_all['stacking'] #one-hot encoded stacking data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [26]:
#fit and evaluate baseline unoptimized random forest model
from sklearn.ensemble import RandomForestClassifier
sklearn_model = RandomForestClassifier(n_estimators=100)
sklearn_model.fit(X_train, y_train)
y_pred=sklearn_model.predict(X_test)
# Evaluate it.
print('f1 score for predicting stacking')
print(classification_report(y_test, y_pred))


f1 score for predicting stacking
              precision    recall  f1-score   support

           0       0.50      0.10      0.16       125
           1       0.87      0.98      0.93       796

    accuracy                           0.86       921
   macro avg       0.69      0.54      0.54       921
weighted avg       0.82      0.86      0.82       921



**Hyperparameter Optimization**

In [0]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

In [0]:
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [29]:
rf_random = RandomizedSearchCV(estimator = sklearn_model, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  2.1min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators=100,
                                                    n_jobs=None,
 

In [31]:
y_true, y_pred = y_test , rf_random.predict(X_test)
print('Results on the test set:')
print(classification_report(y_true, y_pred))

Results on the test set:
              precision    recall  f1-score   support

           0       0.58      0.15      0.24       125
           1       0.88      0.98      0.93       796

    accuracy                           0.87       921
   macro avg       0.73      0.57      0.58       921
weighted avg       0.84      0.87      0.84       921



# Stochastic Gradient Descent (SGD) Classifier Model

In [32]:
#fit and evaluate unoptimized SGD model
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss="log", penalty="l2", max_iter=100, shuffle=True)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
print('f1 score for predicting stacking')
print(classification_report(y_test, y_pred))

f1 score for predicting stacking
              precision    recall  f1-score   support

           0       0.35      0.26      0.30       125
           1       0.89      0.92      0.91       796

    accuracy                           0.83       921
   macro avg       0.62      0.59      0.60       921
weighted avg       0.82      0.83      0.82       921



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


**Hyperparameter Optimization**

In [33]:
loss = ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron']
penalty = ['l1', 'l2', 'elasticnet']
alpha = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
learning_rate = ['constant', 'optimal', 'invscaling', 'adaptive']
class_weight = [{1:0.5, 0:0.5}, {1:0.4, 0:0.6}, {1:0.6, 0:0.4}, {1:0.7, 0:0.3}]
eta0 = [1, 10, 100]

param_distributions = dict(loss=loss,
                           penalty=penalty,
                           alpha=alpha,
                           learning_rate=learning_rate,
                           class_weight=class_weight,
                           eta0=eta0)
random = RandomizedSearchCV(estimator=clf,
                            param_distributions=param_distributions,
                            scoring=None,
                            verbose=1, n_jobs=-1,
                            n_iter=10)
random_result = random.fit(X_train, y_train)

print('Best Score: ', random_result.best_score_)
print('Best Params: ', random_result.best_params_)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best Score:  0.8681881695388914
Best Params:  {'penalty': 'l1', 'loss': 'squared_hinge', 'learning_rate': 'constant', 'eta0': 10, 'class_weight': {1: 0.5, 0: 0.5}, 'alpha': 1000}


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    4.3s finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [34]:
#evaluation of the optimized model
y_true, y_pred = (y_test) , random.predict(X_test)
print('Results on the test set:')
print(classification_report(y_true, y_pred))

Results on the test set:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       125
           1       0.86      1.00      0.93       796

    accuracy                           0.86       921
   macro avg       0.43      0.50      0.46       921
weighted avg       0.75      0.86      0.80       921



# SASA Models

In [0]:
#load the SASA dataset into a Pandas dataframe
import pandas as pd
import io
import requests
url="https://drive.google.com/uc?id=1Y3Imx-lTjGKCQAFqEKTbaMSzFARtwEFN"
s=requests.get(url).content
sasa=pd.read_csv(io.StringIO(s.decode("utf-8")), sep=" ")

In [0]:
sasa

,id,resname,resid,sasa-All-atoms,sasa-Total-Side,sasa-Main-Chain,sasa-Non-polar,sasa-All-polar
0,1A60,GUA,1,134.15,73.92,60.23,31.95,102.20
1,1A60,GUA,2,172.00,56.91,115.09,38.57,133.43
2,1A60,GUA,3,187.59,58.90,128.68,52.70,134.89
3,1A60,ADE,4,186.44,56.18,130.26,57.02,129.43
4,1A60,GUA,5,204.31,87.69,116.62,37.60,166.72
...,...,...,...,...,...,...,...,...
3069,5WQ1,URA,19,179.58,63.37,116.20,50.90,128.68
3070,5WQ1,GUA,20,182.04,56.29,125.76,44.80,137.24
3071,5WQ1,URA,21,177.15,57.36,119.79,51.42,125.72
3072,5WQ1,CYT,22,178.28,54.11,124.17,53.80,124.48


**Model to predict sasa all atoms**

In [0]:
#define training features and target, split the dataset into train/test 
from sklearn.model_selection import train_test_split
X=sasa[['sasa-Total-Side','sasa-Main-Chain', 'sasa-Non-polar', 'sasa-All-polar']] 
y=sasa['sasa-All-atoms'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [37]:
#unoptimized random forest regression model train and evaluate
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
sklearn_model = RandomForestRegressor(n_estimators=100)
sklearn_model.fit(X_train, y_train)
y_pred=sklearn_model.predict(X_test)
# Evaluate it.
print('R2 score for sasa all')
print(sklearn_model.score(X_test, y_test))
print(r2_score(y_test,y_pred))

R2 score for sasa all
0.9951311291543415
0.9951311291543414


**Model to Predict sasa total for side chains**

In [0]:
#define training features and target, split the dataset into train/test 
from sklearn.model_selection import train_test_split
X=sasa[['sasa-All-atoms','sasa-Main-Chain', 'sasa-Non-polar', 'sasa-All-polar']] 
y=sasa['sasa-Total-Side'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [39]:
#unoptimized random forest regression model train and evaluate
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
sklearn_model = RandomForestRegressor(n_estimators=100)
sklearn_model.fit(X_train, y_train)
y_pred=sklearn_model.predict(X_test)
# Evaluate it.
print('R2 score for sasa sidechains')
print(sklearn_model.score(X_test, y_test))
print(r2_score(y_test,y_pred))

R2 score for sasa sidechains
0.9867762563664119
0.9867762563664119


**Model to predict Main Chain sasa**

In [0]:
#define training features and target, split the dataset into train/test
from sklearn.model_selection import train_test_split
X=sasa[['sasa-All-atoms','sasa-Total-Side', 'sasa-Non-polar', 'sasa-All-polar']] 
y=sasa['sasa-Main-Chain'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [42]:
#unoptimized random forest regression model train and evaluate
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
sklearn_model = RandomForestRegressor(n_estimators=100)
sklearn_model.fit(X_train, y_train)
y_pred=sklearn_model.predict(X_test)
# Evaluate it.
print('R2 score for sasa main chain')
print(sklearn_model.score(X_test, y_test))
print(r2_score(y_test,y_pred))

R2 score for sasa main chain
0.9598345179305835
0.9598345179305835


**Model to predict non-polar sasa**

In [0]:
#define training features and target, split the dataset into train/test
from sklearn.model_selection import train_test_split
X=sasa[['sasa-All-atoms','sasa-Total-Side', 'sasa-Main-Chain', 'sasa-All-polar']] 
y=sasa['sasa-Non-polar'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [44]:
#unoptimized random forest regression model train and evaluate
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
sklearn_model = RandomForestRegressor(n_estimators=100)
sklearn_model.fit(X_train, y_train)
y_pred=sklearn_model.predict(X_test)
# Evaluate it.
print('R2 score for sasa non-polar')
print(sklearn_model.score(X_test, y_test))
print(r2_score(y_test,y_pred))

R2 score for sasa non-polar
0.9733997247443472
0.9733997247443472


**Model to predict sasa for all polar residues**

In [0]:
#define training features and target, split the dataset into train/test
from sklearn.model_selection import train_test_split
X=sasa[['sasa-All-atoms','sasa-Total-Side', 'sasa-Main-Chain', 'sasa-Non-polar']] 
y=sasa['sasa-All-polar'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [46]:
#unoptimized random forest regression model train and evaluate
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
sklearn_model = RandomForestRegressor(n_estimators=100)
sklearn_model.fit(X_train, y_train)
y_pred=sklearn_model.predict(X_test)
# Evaluate it.
print('R2 score for sasa polar')
print(sklearn_model.score(X_test, y_test))
print(r2_score(y_test,y_pred))

R2 score for sasa polar
0.987626360676706
0.987626360676706
